In [176]:
# Import the packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk(''):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
import seaborn as sns

In [177]:
#importing data with ';' delimiter
df=pd.read_csv('500richest_BBI/500richestpeople2021_clean.csv', sep=';') 

In [178]:
# check the head of the dataframe
# We see unused columns in 'Unnamed 7' - 'Unnamed 10'
df.head()

,Rank,Name,Total Net Worth,$ Last Change,$ YTD Change,Country,Industry,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1.0,Jeff Bezos,$188B,+$1.68B,-$2.31B,United States,Technology,NaN,NaN,NaN,NaN
1,2.0,Elon Musk,$170B,-$2.89B,+$773M,United States,Technology,NaN,NaN,NaN,NaN
2,3.0,Bernard Arnault,$155B,+$892M,+$40.9B,France,Consumer,NaN,NaN,NaN,NaN
3,4.0,Bill Gates,$144B,-$1.32B,+$12.2B,United States,Technology,NaN,NaN,NaN,NaN
4,5.0,Mark Zuckerberg,$114B,+$203M,+$10.9B,United States,Technology,NaN,NaN,NaN,NaN


In [179]:
#count null fields in df
df.isnull().sum()

Rank                 4
Name                 4
Total Net Worth      4
$ Last Change        4
$ YTD Change         4
Country              4
Industry             4
Unnamed: 7         503
Unnamed: 8         503
Unnamed: 9         503
Unnamed: 10        503
dtype: int64

In [180]:
# We can see that the columns 'Unnamed 7-10' are null and the rows of index 499-502 are null
df.isnull()

,Rank,Name,Total Net Worth,$ Last Change,$ YTD Change,Country,Industry,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,False,False,False,False,False,False,False,True,True,True,True
1,False,False,False,False,False,False,False,True,True,True,True
2,False,False,False,False,False,False,False,True,True,True,True
3,False,False,False,False,False,False,False,True,True,True,True
4,False,False,False,False,False,False,False,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
498,False,False,False,False,False,False,False,True,True,True,True
499,True,True,True,True,True,True,True,True,True,True,True
500,True,True,True,True,True,True,True,True,True,True,True
501,True,True,True,True,True,True,True,True,True,True,True


<h1>Data Cleaning</h1>
<h4>1. Drop unnecessary columns</h4>

In [181]:
to_drop = ['Unnamed: 7',
          'Unnamed: 8',
          'Unnamed: 9',
          'Unnamed: 10']
#inplace=True means we're working on the original object
#axis=1 means we're dropping the column, not row
df.drop(columns=to_drop, inplace=True)

<h4>2. Drop necessary rows</h4>

In [182]:
df.drop([499, 500, 501, 502], inplace=True)

In [183]:
df.isnull().sum()

Rank               0
Name               0
Total Net Worth    0
$ Last Change      0
$ YTD Change       0
Country            0
Industry           0
dtype: int64

In [184]:
df.dtypes

Rank               float64
Name                object
Total Net Worth     object
$ Last Change       object
$ YTD Change        object
Country             object
Industry            object
dtype: object

<h5>'Total Net Worth', '\$ Last Change', and '\$ YTD Change' are all objects. We want float for our EDA</h5>

In [185]:
# df['Total Net Worth'] = [x.strip('$') for x in df['Total Net Worth']]
df[df.columns[2:5]] = df[df.columns[2:5]].replace('[\$,]', '', regex=True)

In [186]:
df.head()

,Rank,Name,Total Net Worth,$ Last Change,$ YTD Change,Country,Industry
0,1.0,Jeff Bezos,188B,+1.68B,-2.31B,United States,Technology
1,2.0,Elon Musk,170B,-2.89B,+773M,United States,Technology
2,3.0,Bernard Arnault,155B,+892M,+40.9B,France,Consumer
3,4.0,Bill Gates,144B,-1.32B,+12.2B,United States,Technology
4,5.0,Mark Zuckerberg,114B,+203M,+10.9B,United States,Technology


In [187]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return 0.0

df['Total Net Worth'] = df['Total Net Worth'].apply(value_to_float)
df['$ Last Change'] = df['$ Last Change'].apply(value_to_float)
df['$ YTD Change'] = df['$ YTD Change'].apply(value_to_float)

In [188]:
df[df.columns[2:5]].head()

,Total Net Worth,$ Last Change,$ YTD Change
0,1.880000e+11,1.680000e+09,-2.310000e+09
1,1.700000e+11,-2.890000e+09,7.730000e+08
2,1.550000e+11,8.920000e+08,4.090000e+10
3,1.440000e+11,-1.320000e+09,1.220000e+10
4,1.140000e+11,2.030000e+08,1.090000e+10


In [189]:
df.dtypes

Rank               float64
Name                object
Total Net Worth    float64
$ Last Change      float64
$ YTD Change       float64
Country             object
Industry            object
dtype: object

<h4>3. Identifying outliers</h4>

In [190]:
df.describe()

,Rank,Total Net Worth,$ Last Change,$ YTD Change
count,499.000000,4.990000e+02,4.990000e+02,4.990000e+02
mean,250.192385,1.619824e+10,-6.275872e+07,1.192092e+09
std,144.462829,2.029522e+10,1.038117e+09,4.179780e+09
min,1.000000,0.000000e+00,-2.890000e+09,-1.830000e+10
25%,125.500000,7.170000e+09,-1.805000e+08,-4.925000e+06
50%,250.000000,9.760000e+09,-7.050000e+07,7.200000e+08
75%,374.500000,1.580000e+10,-7.135000e+06,1.685000e+09
max,500.000000,1.880000e+11,2.110000e+10,4.090000e+10


<h5>We can see that the minimum 'Total Net Worth' is 0.0. That doesn't make any sense.</h5>

In [191]:
# find the index of the min value in 'Total Net Worth'
df[['Total Net Worth']].idxmin()

Total Net Worth    84
dtype: int64

In [192]:
df.iloc[83:86]

,Rank,Name,Total Net Worth,$ Last Change,$ YTD Change,Country,Industry
83,84.0,Stefan Quandt,2.140000e+10,-2.270000e+08,2.600000e+09,Germany,Industrial
84,85.0,Sara Mota de Larrea & Family,0.000000e+00,2.110000e+10,6.550000e+07,Mexico,Commodities
85,86.0,Alisher Usmanov,2.090000e+10,1.600000e+08,-1.920000e+08,Russia,Diversified


In [193]:
# The outlier looks like a simple mistake. Replace with data in row index 38 from the Bloomberg Billionaire Index to date (June 10, 2021)
df.iloc[[84],2:5] = 18.7e+10,1.96e+8,1.89e+10

In [194]:
df.iloc[83:86]

,Rank,Name,Total Net Worth,$ Last Change,$ YTD Change,Country,Industry
83,84.0,Stefan Quandt,2.140000e+10,-227000000.0,2.600000e+09,Germany,Industrial
84,85.0,Sara Mota de Larrea & Family,1.870000e+11,196000000.0,1.890000e+10,Mexico,Commodities
85,86.0,Alisher Usmanov,2.090000e+10,160000000.0,-1.920000e+08,Russia,Diversified


In [195]:
df.describe()

,Rank,Total Net Worth,$ Last Change,$ YTD Change
count,499.000000,4.990000e+02,4.990000e+02,4.990000e+02
mean,250.192385,1.657299e+10,-1.046505e+08,1.229836e+09
std,144.462829,2.167508e+10,4.204049e+08,4.253968e+09
min,1.000000,5.720000e+09,-2.890000e+09,-1.830000e+10
25%,125.500000,7.170000e+09,-1.805000e+08,-4.925000e+06
50%,250.000000,9.770000e+09,-7.050000e+07,7.310000e+08
75%,374.500000,1.590000e+10,-7.135000e+06,1.705000e+09
max,500.000000,1.880000e+11,4.600000e+09,4.090000e+10


<h1>Exploratory Data Analysis</h1>